# Matrix multiplication

In [1]:
import nbloader
import tensorflow as tf
import numpy as np
from demo_util import gpusession
print ("Packages loaded.")
print 'TF version: '+tf.__version__

Packages loaded.
TF version: 1.3.0


### Checking tensor operations 
- For each N data in a batch
    - Sample K weight matrices whose size is [Q x D]
- Weight sampler: [N x D x K x Q]

In [3]:
# Configuration
Q = 3 # Input dimension
D = 2 # Output dimension
K = 4 # Number of mixture

# Make Graph
# Input placeholder and reshape
X = tf.placeholder(dtype=tf.float32,shape=[None,Q]) # Input [N x Q]
nX,dimX = tf.shape(X)[0],tf.shape(X)[1] # Get size of X, (nX=N, dimX=Q)
Xrsh = tf.reshape(X,shape=[nX,dimX,1]) # [N x Q x 1]

# Random sampler (this will work as BBB.)
# Sample K times using sampler
samplerList = []
for sIdx in range(K): # Make a list of K samplers 
    PZ = tf.random_normal(shape=[nX,Q,D],mean=0,stddev=1,dtype=tf.float32) # [N x Q x D]
    QZ = tf.random_normal(shape=[nX,Q,D],mean=0,stddev=1,dtype=tf.float32) # [N x Q x D]
    samplerList.append(PZ+QZ)
WlistConcat = tf.convert_to_tensor(samplerList) # [K x N x Q x D]
WlistConcatTr = tf.transpose(WlistConcat,perm=[1,3,0,2]) # [N x D x K x Q]

# Matrix multiplication
WlistConcatTrRsh = tf.reshape(WlistConcatTr,shape=[nX,K*D,dimX]) # [N x KD x Q]
mu = tf.matmul(WlistConcatTrRsh,Xrsh) # [N x KD x Q] x [N x Q x 1] => [N x KD x 1]
muRsh = tf.reshape(mu,shape=[nX,D,K]) # [N x D x K]

# Run
N = 5 # Batch size
Xval = np.random.rand(N,Q) # Random batch inputs
sess = gpusession() # Open session
WlistConcatTrVar,muRshVal \
    = sess.run([WlistConcatTr,muRsh],feed_dict={X:Xval}) # [N x K x D x Q] / [N x D x K]

# Check whether the computed 'muRsh' [N x D x K]
V = np.zeros(shape=[N,D,K])
for nIdx in range(N): # For all batch
    cX = Xval[nIdx:nIdx+1,:] # [1 x Q]
    cW = WlistConcatTrVar[nIdx,:,:,:].squeeze() # [D x K x Q]
    for jIdx in range(K): # For each mixture
        cWj = cW[:,jIdx,:].squeeze() # [D x Q] 
        cWjT = cWj.T # [Q x D] - Input to output mapping
        cVj = np.matmul(cX,cWjT) # [1 x Q] x [Q x D] => [1 x D]
        V[nIdx,:,jIdx] = cVj # [1 x D]
# Check error
err = np.linalg.norm(muRshVal-V)
print ('err: %.3e'%(err))

err: 2.534e-07
